In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
import pandas as pd
#df = pd.read_csv('spam.csv')
#df.head(5)
df= pd.read_csv('https://gist.githubusercontent.com/mydpy/5fb434b1014a6039502faa1e6b6e1d05/raw/0fda71e716fc8105cfe40c74ec33fe63b5f61382/SMSSpamCollection.csv',
                           names=["label", "message"])
df.head()

,label,message
0,label,text
1,0.0,Go until jurong point
2,0.0,Ok lar... Joking wif u oni...
3,1.0,Free entry in 2 a wkly comp to win FA Cup fina...
4,0.0,U dun say so early hor... U c already then say...


In [ ]:
df.rename(columns = {'label':'Category', 'message':'Message'}, inplace = True)
df.head()

,Category,Message
0,label,text
1,0.0,Go until jurong point
2,0.0,Ok lar... Joking wif u oni...
3,1.0,Free entry in 2 a wkly comp to win FA Cup fina...
4,0.0,U dun say so early hor... U c already then say...


In [ ]:
df.groupby('Category').describe()

Message                                            
           count unique                             top freq
Category                                                    
0.0         4823   4339                           Sorry   52
1.0          747    623  Thanks for your ringtone order    5
label          1      1                            text    1

In [ ]:
df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,label,text,0
1,0.0,Go until jurong point,0
2,0.0,Ok lar... Joking wif u oni...,0
3,1.0,Free entry in 2 a wkly comp to win FA Cup fina...,0
4,0.0,U dun say so early hor... U c already then say...,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Message'],df['spam'], stratify=df['spam'])

In [ ]:
X_train.head(4)

3675    Moji just informed me that you saved our lives...
775     I wil be there with in  ltgt  minutes. Got any...
2096    PRIVATE Your 2004 Account Statement for 077426...
2740    I sent you the prices and do you mean the  ltgt g
Name: Message, dtype: object

In [ ]:
!pip install tensorflow-text

In [ ]:
import tensorflow_text as text

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435169 , -0.5132726 , -0.8884571 , ..., -0.7474883 ,
        -0.75314724,  0.91964495],
       [-0.8720837 , -0.50543964, -0.94446695, ..., -0.858475  ,
        -0.7174535 ,  0.88082975]], dtype=float32)>

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [ ]:
X_train

3675    Moji just informed me that you saved our lives...
775     I wil be there with in  ltgt  minutes. Got any...
2096    PRIVATE Your 2004 Account Statement for 077426...
2740    I sent you the prices and do you mean the  ltgt g
568           Oooh bed ridden ey What are YOU thinking of
                              ...                        
4642    Do you hide anythiing or keeping distance from me
330                                                  Cool
13      URGENT You have won a 1 week FREE membership i...
434     Congrats Nokia 3650 video camera phone is your...
3942                            Does daddy have a bb now.
Name: Message, Length: 4181, dtype: object

In [ ]:
#X_train=get_sentence_embeding(X_train)

In [ ]:
#import numpy as np
#X_train = np.asarray(X_train).astype(np.float32)

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2, batch_size = 32)

ValueError: ignored

In [ ]:
model.evaluate(X_test, y_test)